# Tiền xử lý dữ liệu

## Tại sao cần phải tiền xử lý dữ liệu
Tiền xử lý dữ liệu là một phần quan trọng trong quy trình phân tích dữ liệu và xây dựng mô hình máy học, giúp cải thiện chất lượng và hiệu suất của mô hình cuối cùng.

## Mục tiêu
Ở phần này, dữ liệu chúng ta thu được là dữ liệu dạng **văn bản**. Tuy nhiên các mô hình học máy chỉ có thể "học" khi dữ liệu là dạng **số thực** nên trong phần này, việc quan trọng nhất đó chính là *Làm thế nào để chuyển dữ liệu sang dạng số thực?* 

## Import các thư viện cần dùng

In [21]:
import os
import pickle

import string

import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm

import plotly.express as px

import gensim
from pyvi import ViTokenizer

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
np.random.seed(42)
warnings.filterwarnings("ignore")

DATA_PATH = "../data/"
MODEL_PATH = "../models/"

### Đọc dữ liệu từ thư mục data/
Vì dữ liệu được lưu dưới dạng JSON, nên chúng ta có thể sử dụng trực tiếp phương thức `read_json()` trong thư viện `pandas`

In [3]:
def load_data(model_path: str, filename: str):
    data = pickle.load(open(os.path.join(model_path, filename), "rb"))
    return data

In [4]:
X_train = load_data(MODEL_PATH, "X_train.pkl")
X_test = load_data(MODEL_PATH, "X_test.pkl")
y_train = load_data(MODEL_PATH, "y_train.pkl")
y_test = load_data(MODEL_PATH, "y_test.pkl")

In [5]:
le = load_data(MODEL_PATH, 'label_encoder.pkl')

In [6]:
print(f"Train set has {X_train.shape[0]} observations")
print(f"Test set has {X_test.shape[0]} observations")

Train set has 11144 observations
Test set has 2787 observations


## Tiền xử lý dữ liệu

In [7]:
data = pd.DataFrame({
    'content': X_train,
    'category': y_train
})

### Loại trừ các dữ liệu NaN

In [8]:
data.isnull().sum()

content     0
category    0
dtype: int64

In [9]:
data.head()

,content,category
0,"Haruki Murakami, 74 tuổi, là tiểu thuyết gia c...",12
1,"Sáng ngày 5/4, hàng loạt fanpage tại Việt Nam ...",8
2,"Nguyễn Thị Hậu sinh năm 2005 ở Thường Xuân, Th...",2
3,Đây cũng là công tác góp phần thực hiện hiệu q...,11
4,Sự vượt trội của hai đội tuyển đã được thể hiệ...,10


Dữ liệu trên, không có theo dõi nào có thuộc tính bị NaN hoặc rỗng, do đó, ở phần này chúng ta không cần xử lý phần việc loại trừ các giá trị bị rỗng

In [10]:
count = data["category"].value_counts()
num_of_post_each_topic = pd.DataFrame(
    {"category": count.index, "count": count.values}
)

In [11]:
le.classes_

array(['An sinh', 'Bất động sản', 'Giáo dục', 'Giải trí', 'Kinh doanh',
       'Nhân ái', 'Pháp luật', 'Sức khỏe', 'Sức mạnh số', 'Thế giới',
       'Thể thao', 'Việc làm', 'Văn hóa', 'Xe ++', 'Xã hội'], dtype=object)

In [12]:
num_of_post_each_topic['category'] = le.inverse_transform(num_of_post_each_topic['category'])
num_of_post_each_topic

,category,count
0,Thể thao,1197
1,Giáo dục,1158
2,Sức khỏe,1026
3,Bất động sản,977
4,Kinh doanh,898
5,Văn hóa,832
6,Xã hội,767
7,Xe ++,727
8,Sức mạnh số,629
9,Thế giới,602


In [13]:
fig = px.histogram(
    num_of_post_each_topic,
    x='category',
    y='count',
    color='category',
    title="Số lượng bài báo trên từng hạng mục",
    color_discrete_sequence=px.colors.qualitative.Set1,
)
fig.update_yaxes(showgrid=True)  # the y-axis is in dollars
fig.update_layout(  # customize font and legend orientation & position
    font_family="Arial",
    legend=dict(
        title=None, orientation="h", y=1, yanchor="bottom", x=0.5, xanchor="center"
    ),
    yaxis_title="Số lượng bài báo",
    xaxis_title="Hạng mục",
)

fig.add_shape(  # add a horizontal "target" line
    type="line",
    line_color="#16FF32",
    line_width=3,
    opacity=1,
    line_dash="dot",
    x0=0,
    x1=1,
    xref="paper",
    y0=929,
    y1=929,
    yref="y",
)

fig.add_annotation(  # add a text callout with arrow
    text="Số lượng ít nhất", x="Pháp luật", y=360, arrowhead=1, showarrow=True
)


fig.show()

> Nhận thấy rằng, dữ liệu trên đang ở trong tình trạng mất cân bằng (**Imbalanced data**) khi mà số lượng bài báo thuộc hạng mục **Thể thao** gấp gần 4 lần so với bài báo thuộc thể loại **Pháp luật**

### Building Vocabulary and Cleaning Text

In [14]:
# Get punctuation list
punc = string.punctuation
# Get digits list
digits = string.digits

# Read vietnamese-stopwords.txt in data/
with open("../data/vietnamese-stopwords.txt", encoding='UTF-8') as f:
    lines = [line.rstrip("\n").strip() for line in f]

stopwords = [line.replace(' ', '_') for line in lines]
punc_custom = punc.translate(str.maketrans("", "", "_"))
stopwords[:5]

['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy']

In [16]:
X_train.shape

(11144,)

In [22]:
class TextCleaner(BaseEstimator, TransformerMixin):
    def __init__(self) -> None:
        super().__init__()
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        cleaned_texts = []
        for text in X:
            cleaned_text = self.clean_text(text)
            cleaned_texts.append(cleaned_text)
        return np.array(cleaned_texts)
    
    @staticmethod
    def clean_text(content: str):
        # # lowercase content
        # content = content.lower()
        # # remove white space
        # content = " ".join([x for x in content.split()])
        # # remove punctuation and digits
        # content = content.translate(str.maketrans("", "", punc_custom)).translate(
        #     str.maketrans("", "", digits)
        # )
        tokens = gensim.utils.simple_preprocess(content)
        content = " ".join(tokens)
        # tokenize content
        content = ViTokenizer.tokenize(content)
        return content

In [23]:
# Chỉ giữ lại 90% từ vựng để biểu diễn
count_vectorizer = CountVectorizer(stop_words=stopwords)
text_cleaner = TextCleaner()
model_pipeline = Pipeline(
    [
        ('text_cleaner', text_cleaner),
        ("count_vectorizer", count_vectorizer),
        ("tfidf_transformer", TfidfTransformer()),
    ]
)

model_pipeline.fit(X_train, y_train)

# data_preprocessed = model_pipeline.fit_transform(X_train, y_train)
# print(f"Số chiều của dữ liệu: {data_preprocessed.shape}")
print(f"Số từ trong từ điển: {len(count_vectorizer.vocabulary_)}")

Số từ trong từ điển: 59231


In [26]:
tokens = gensim.utils.simple_preprocess(X_train[0])

In [28]:
content = ' '.join(tokens)

In [29]:
ViTokenizer.tokenize(content)

'haruki murakami tuổi là tiểu_thuyết gia có sách bán_chạy nhất tại nhật bản ông bắt_đầu sự_nghiệp viết_lách năm_tuổi và trở_thành một hiện_tượng văn_học vào năm khi cuốn tiểu_thuyết thứ rừng na uy được xuất_bản sự pha_trộn giữa những câu_chuyện thực_tế và mơ_mộng của haruki murakami đã mang đến cho ông một lượng người hâm_mộ đông_đảo tên của ông thường nổi lên như một ứng_cử_viên cho giải_thưởng nobel văn_học chân_dung haruki murakami thiết_kế the guardian sách cho người mới bắt_đầu_đọc tiểu_thuyết của murakami có_thể được chia thành loại kỳ_ảo và hiện_thực nhiều cuốn nằm giữa thể_loại này được xuất_bản năm rừng na uy là một hồi_ức đơn_giản về tình_yêu tuổi_trẻ hạ_cánh trên một đường_băng đức người kể chuyện toru watanabe nghe bài hát nổi_tiếng của the beatles cảm_tưởng như trở_lại những năm_tháng sinh_viên và mối tình đầy sóng_gió với cô gái hoài_cổ và ngọt_ngào rừng na uy là cuốn tiểu_thuyết dễ tiếp_cận nhất của murakami và là cuốn sách đã biến tác_giả thành một siêu_sao văn_học nhật